In [47]:
import csv
import glob
from datetime import datetime
from collections import deque
import pandas as pd
import json
import decimal


decimal.getcontext().prec = 6

with open('conf.json', encoding='utf8') as cf:
    param = json.load(cf)

index = ['temp','humi','time','weather']

files = glob.glob("CSV/*.csv")

#csv2フォルダに入るファイル
csv2 = []

with open(files[0], 'r', encoding='ansi') as f:
    while True:
        line = f.readline()
        if not line:
            break
        items = line.replace('\n','').split(',')
        csv2.append(
            decimal.Decimal(items[3]) * decimal.Decimal(param["Humid2Ratio"])
        )
        
        

In [46]:
csv2

[Decimal('0.9200000'),
 Decimal('0.9400000'),
 Decimal('0.9200000'),
 Decimal('0.9200000'),
 Decimal('0.9400000'),
 Decimal('0.9400000'),
 Decimal('0.9400000'),
 Decimal('0.9400000'),
 Decimal('0.9600000'),
 Decimal('0.9600000'),
 Decimal('0.9600000')]

In [44]:
import decimal
decimal.getcontext().prec = 7
a = decimal.Decimal(5.555555555555)
b = decimal.Decimal(1)
print(a*b)


5.555556


In [ ]:


#CSV形式にて、[元画像ファイル名,出力画像ファイル名,縦横判定,白黒判定,線幅 X 10]
csvdata = []

for file in files:
    pass


timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

df = pd.DataFrame(csvdata)
df.to_csv(f'after/result{timestamp}.csv')


#1行格納しながら変更してしまう
    #
#処理中の時間取得{7:1}
#次の行の格納をしながら変更
#処理中の時間と比較
    #同じ=>継続{7:2}
    #違う=>不足数を確認
        #不足数分のステップデータを作成
        #処理中の時間取得{8:1}

    #データがもうない=>不足数を確認
        #不足数分のステップデータを作成



In [ ]:
import pandas as pd
from datetime import datetime
from collections import deque

def read_log(path):

    cams = dict() # キー=カメラ, 値=日時リスト

    with open(path, 'r', encoding='shift_jis', errors='ignore') as f:
        read_state = 0
        que = deque(maxlen=5)
        while True:
            line = f.readline()
            if not line:
                break

            que.append(line) # いったん流れた日時行を取得するため直近5行をキューに保持

            if read_state == 0:   # 開始行の探索中
                if line.startswith('サブリスト: 動体検知開始'):
                    line = que[-4] # 日時行
                    pos = line.find(' ')
                    dt = datetime.strptime(line[pos+1:].strip(), '%Y-%m-%d %H:%M:%S')
                    read_state = 1

            elif read_state == 1: # カメラ行の探索中
                if line.startswith('カメラ No.:'):
                    cam = line.split(':')[1].strip()
                    if cam not in cams:
                        cams[cam] = []
                    cams[cam].append(dt)
                    read_state = 0

    return cams

cams = read_log('logBack.txt')
print('read_log end.')

# 結果をカメラ毎にCSV出力
for cam, dts in cams.items():
    df = pd.DataFrame({'Date':dts})
    df['cnt'] = 1
    df = df.set_index('Date')
    df = df.resample('H').sum()
    df.to_csv(f'{cam}.csv')